In [52]:
import pandas as pd
import parsel
import requests

In [4]:
url_base = 'https://sga.softexrecife.org.br'

In [9]:
endpoints_catalogo = ['/catalogo?page=1']

while True:
    resposta =  requests.get(f'{url_base}{endpoints_catalogo[-1]}')
    html_resposta = resposta.text
    html_selector = parsel.Selector(html_resposta)
    next_endpoint = html_selector.xpath("//li[@class='next']/a/@href").get()
    if next_endpoint:
        print(next_endpoint)
        endpoints_catalogo.append(next_endpoint)
    else:
        print(next_endpoint)
        break

/catalogo?page=2
/catalogo?page=3
/catalogo?page=4
/catalogo?page=5
/catalogo?page=6
/catalogo?page=7
/catalogo?page=8
/catalogo?page=9
/catalogo?page=10
/catalogo?page=11
/catalogo?page=12
/catalogo?page=13
/catalogo?page=14
/catalogo?page=15
/catalogo?page=16
/catalogo?page=17
/catalogo?page=18
None


In [36]:
links = []

for endpoints in endpoints_catalogo:
    link_requisicao = f'{url_base}{endpoints}'
    links.append(link_requisicao)

links
    

['https://sga.softexrecife.org.br/catalogo?page=1',
 'https://sga.softexrecife.org.br/catalogo?page=2',
 'https://sga.softexrecife.org.br/catalogo?page=3',
 'https://sga.softexrecife.org.br/catalogo?page=4',
 'https://sga.softexrecife.org.br/catalogo?page=5',
 'https://sga.softexrecife.org.br/catalogo?page=6',
 'https://sga.softexrecife.org.br/catalogo?page=7',
 'https://sga.softexrecife.org.br/catalogo?page=8',
 'https://sga.softexrecife.org.br/catalogo?page=9',
 'https://sga.softexrecife.org.br/catalogo?page=10',
 'https://sga.softexrecife.org.br/catalogo?page=11',
 'https://sga.softexrecife.org.br/catalogo?page=12',
 'https://sga.softexrecife.org.br/catalogo?page=13',
 'https://sga.softexrecife.org.br/catalogo?page=14',
 'https://sga.softexrecife.org.br/catalogo?page=15',
 'https://sga.softexrecife.org.br/catalogo?page=16',
 'https://sga.softexrecife.org.br/catalogo?page=17',
 'https://sga.softexrecife.org.br/catalogo?page=18']

In [19]:
def retorna_seletor(link): 
    resposta = requests.get(link)
    html_resposta = resposta.text
    html_selector = parsel.Selector(html_resposta)

    return html_selector


In [27]:
def buscar_cards(seletor):
    links_cards = seletor.xpath("//div[@class='catalog-content-main-company border cl-page-width rounded row d-flex align-items-center justify-content-center']/a/@href").getall()

    return links_cards

In [42]:
def dados_startup(link):
    try:
        cards_selector = retorna_seletor(link)

        telefone = cards_selector.xpath("//div[@class='col-contact']//div[@class='col-content']/a[1]/text()").get()
        email = cards_selector.xpath("//div[@class='col-contact']//div[@class='col-content']/a[2]/text()").get()
        site = cards_selector.xpath("//div[@class='col-contact']//div[@class='col-content']/a[3]/text()").get()
        endereco = cards_selector.xpath("//div[@class='col-contact']//div[@class='col-content']/a[4]/text()").get().split()
        endereco = ' '.join(endereco)
        nome_startup = cards_selector.xpath("//div[@class='col-details']/p/text()").get().strip()
        descricao_startup = cards_selector.xpath("//div[@class='col-details']/textarea/text()").get().split()
        descricao_startup = ' '.join(descricao_startup)
        segmento_startup = cards_selector.xpath("//div[@class='col-details']/p[3]/text()").get()
        modelo_startup = cards_selector.xpath("//div[@class='col-details']/p[5]/text()").get()

        dados = {
            'Nome': nome_startup,
            'Segmento': segmento_startup,
            'Modelo': modelo_startup,
            'Site': site,
            'Email': email,
            'Telefone': telefone,
            'Endereço': endereco,
            'Descrição': descricao_startup,
        }

        return dados

                
    except Exception as E:
        print(f'Erro ao acessar o site: {E}, {link}')

In [47]:
dados = []

for i in range(len(links)):    
    seletor = retorna_seletor(links[i])
    links_cards = buscar_cards(seletor)
    for j in range(len(links_cards)):
        card_link =  f'{url_base}{links_cards[j]}'
        dados.append(dados_startup(card_link))
    
dados

[{'Nome': 'ATENDE.UP',
  'Segmento': 'CommunityTech',
  'Modelo': 'SaaS ',
  'Site': 'https://atendeup.com/',
  'Email': 'liviac.atendeup@gmail.com',
  'Telefone': '(81)99816-6331',
  'Endereço': 'Rua Dona Maria Cesar, 170 - sala 0203Recife/PE. CEP: 50030-140.',
  'Descrição': 'A AtendeUp é uma plataforma inovadora que otimiza o atendimento empresarial, reunindo toda a equipe em um único número de WhatsApp. Com uma interface simples e fácil de usar, ela permite gerenciar os contatos de forma eficiente, garantindo respostas rápidas e personalizadas por meio de mensagens automáticas com textos, links, imagens, áudios, vídeos e documentos. Além disso, a AtendeUp possibilita organizar os contatos de maneira inteligente, categorizando-os por setores, etiquetas, canais e empresas, o que aumenta a produtividade e a eficiência no atendimento ao cliente.'},
 {'Nome': 'Septem Compliance Sistemas e Consultoria LTDA',
  'Segmento': 'GovTech',
  'Modelo': 'BPM',
  'Site': 'https://septemcompliance.

In [51]:
df = pd.DataFrame(dados)

df.to_excel('Softex.xlsx', index=False)